## 유사 도서 추천 시스템

[앱 시연](https://nammtaeehyeonn-reco-recommend-utwo4b.streamlit.app/){target=_blank}

- 책 제목을 입력하면 내용이 유사한 도서 5권 추천하는 앱

In [1]:
# calc_similarity.py

import pandas as pd
import numpy as np
from string import punctuation
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv('data/book.csv')

# 제 컴퓨터로는 데이터 개수를 줄여야 돌아갑니다...
df1 = df.head(3000)

description = df1['description']

# TF - IDF 기법으로 벡터화
# 20000개의 줄거리는 약 80000개의 단어로 이루어져있다. (전처리 완료돼서 많이 줄었다.)
tf_idf = TfidfVectorizer()
tf_idf_matrix = tf_idf.fit_transform(description)
print('TF-IDF 행렬의 크기(shape) :',tf_idf_matrix.shape)

# 전체 유사도 계산
cos_sim = cosine_similarity(tf_idf_matrix, tf_idf_matrix)
print('코사인 유사도 연산 결과 :',cos_sim.shape)

np.save('data/sim.npy', cos_sim)

TF-IDF 행렬의 크기(shape) : (2999, 30145)
코사인 유사도 연산 결과 : (2999, 2999)


In [ ]:
# recommend.py

import streamlit as st
import numpy as np
import pandas as pd

def Recommend(title, cos_sim):
    
    title_idx = dict(zip(df['Title'], df.index))
    # 도서 제목 입력하면 인덱스를 리턴
    idx = title_idx[title]

    # 도서 줄거리 유사도 전부 가져오기
    sim = list(enumerate(cos_sim[idx]))

    # 유사도에 따라 정렬하기
    sim = sorted(sim, key=lambda x: x[1], reverse=True)

    # 유사도 탐5 가져오기
    sim = sim[1:6]

    # 유사도 탑5 인덱스 가져오기
    rec_idx = [idx[0] for idx in sim]

    # 유사도 탑10 제목 가져오기
    # print(sim)                       # (인덱스, 유사도)
    return df.iloc[rec_idx]

df = pd.read_csv('book.csv')
cos_sim = np.load('sim.npy')

# streamlit
st.title('독서는 마음의 양식')

title = st.text_input("책 제목을 입력해주세요")

if title:
    ans = Recommend(title, cos_sim)
    ans.reset_index(drop = True, inplace = True)
     
    for i in range(5):
        st.subheader(ans.Title[i])    # 제목
        col1, col2 = st.columns(2)
        with col1:
            st.image(ans.image[i], width = 200)  # 책 표지
        with col2:
            st.write(ans.description[i][:400] + "   ...")  # 책 내용
        st.header("")

### streamlit에서 앱 작동

<img src='images/Screenshot2022-11-284.44.59.png' width=600px>

## 마르코프 체인을 이용한 챗봇 엔진 만들기

"파이썬을 이용한 머신러닝, 딥러닝 실전 개발 입문(쿠지라 히코우즈쿠에)"의 6장에서 소개한 코드를 변경해서 구현했습니다. 마르코프 체인이 무엇인지에 대해서 설명할 능력은 없습니다. 궁금하면 여기☞ [유튜브 설명](https://www.youtube.com/watch?v=Yh62wN2kMkA)를 눌러 보세요. 

제목에 마르코프 체인을 이용해서 챗봇 엔진을 만들었다고 했지만, 코드를 보면 확률을 사용하지 않고 랜덤하게 다음 단어를 선택했음을 알 수 있습니다. 구체적으로 이 책에서 제공한 코드가 어떻게 작동하는지 아래 2줄의 대화 데이터로 살펴보겠습니다.

In [1]:
text = '''
이번에 캘리포니아에 산불 난 거 보셨어요?
네 봤어요 ㅜㅜ 일주일 넘게 진압하고 있다는데 걱정이네요...
'''

konlpy에서 제공하는 Okt 클래스를 이용해 형태소 분석을 했습니다. 다음은 형태소 분석한 데이터입니다. 

In [2]:
from konlpy.tag import Okt

okt = Okt()
morph_list = okt.pos(text)
print(morph_list)

[('\n', 'Foreign'), ('이번', 'Noun'), ('에', 'Josa'), ('캘리포니아', 'Noun'), ('에', 'Josa'), ('산불', 'Noun'), ('난', 'Noun'), ('거', 'Noun'), ('보셨어요', 'Verb'), ('?', 'Punctuation'), ('\n', 'Foreign'), ('네', 'Noun'), ('봤어요', 'Verb'), ('ㅜㅜ', 'KoreanParticle'), ('일주일', 'Noun'), ('넘게', 'Verb'), ('진압', 'Noun'), ('하고', 'Josa'), ('있다는데', 'Adjective'), ('걱정', 'Noun'), ('이네', 'Josa'), ('요', 'Noun'), ('...', 'Punctuation'), ('\n', 'Foreign')]


형태소 분석한 데이터 중에서 품사명은 제외하고 형태소만을 리스트로 추린 후 3 단어씩 묶었습니다. 이렇게 묶은 이유는 어떤 단어의 뒤에 오는 단어를 2개까지 확보하기 위해서입니다. 특정 단어 뒤에 올만한 단어를 2개까지 묶는 겁니다. '@'는 문장의 처음을 표시하는 기호입니다.

In [3]:
word_list = []
for morph in morph_list:
    # 마침표를 제외한 모든 구두점 제외
    if not morph[1] in ['Punctuation']:
        word_list.append(morph[0])
    if morph[0] == '.':
        word_list.append(morph[0])
print(word_list)

['\n', '이번', '에', '캘리포니아', '에', '산불', '난', '거', '보셨어요', '\n', '네', '봤어요', 'ㅜㅜ', '일주일', '넘게', '진압', '하고', '있다는데', '걱정', '이네', '요', '\n']


이제 데이터를 만드는 마지막 단계입니다. 위에서 만든 리스트를 딕셔너리 자료형으로 바꿉니다. 이 딕셔너리가 문장을 만들 때 어떻게 사용되는지는 다음과 같습니다. 

예를 들어, 
1. 사용자가 '이번(w1)'이라는 단어를 입력하면 '이번'이라는 키에 해당하는 값을 찾습니다. (①)
2. 아래 딕셔너리에는 값이 '에(w2)' 하나뿐입니다. 따라서 '이번'키의 값인 '에'와 '에'키의 값인 '캘리포니아'를 반환합니다. 
3. 다음은 부모로 '에'키와 '캘니포니아'키를 지닌 키를 찾습니다. 여기서는 '에(w3)'네요. (②)
4. 지금까지 만들어진 문장은 '이번 에 캘리포니아 에'입니다.
5. 이어서 부모 키가 '캘리포니아'키와 '에'키인 단어를 찾습니다. '산불'이네요.(③)
6. 그러면 '이번 에 캘리포니아 에 산불'이라는 문장이 만들어집니다.
7. 이런 과정을 반복해 문장을 만들고, 값으로 마침표나 물음표 같은 문장부호, 또는 공백이 나오면 문장 생성을 종료합니다.

여기서는 설명을 위해 문장 2개로 만든 사전의 일부만 사용해서 자료와 똑같은 문장이 만들어졌지만, 많은 문장을 사용하면 하나의 단어키에 해당하는 값이 여러 개가 되고, 그 중에 하나를 랜덤하게 추출하는 것으로 코드가 작성되어 있어 데이터와 다른 문장이 생성됩니다. 랜덤하게 값에 해당하는 형태소를 가져오기 때문에 마르코프 체인의 확률은 구현되지 않은 것 같습니다. 그래서인지 문법이나 뜻이 어색한 문장이 종종 만들어집니다.


```
{
  "@": {
    "이번": {
      "에": 1
    },
    "네": {
      "봤어요": 1
    }
  },
  "이번": {     <------- ①
    "에": {
      "캘리포니아": 1
    }
  },
  "에": {
    "캘리포니아": {
      "에": 1  <------- ②
    },
    "산불": {
      "난": 1
    }
  },
  "캘리포니아": {
    "에": {
      "산불": 1  <------- ③
    }
(이하 생략)
```

In [4]:
dic = {}

tmp = ['@']
for i, word in enumerate(word_list):
    if word == '' or word =='\r\n' or word == '\n': continue

    tmp.append(word)
    if len(tmp) < 3: continue      # 3개 단어씩 묶기
    if len(tmp) > 3: tmp = tmp[1:] # tmp 리스트 길이가 3을 넘어가면 첫 번째 요소 제외
    # print(tmp)
    # sys.exit(0)

    w1, w2, w3 = tmp 
    if not w1 in dic: dic[w1] = {}   # w1 단어가 사전에 없으면 등록
    if not w2 in dic[w1]: dic[w1][w2] = {}  # w2 단어가 w1 블록 안에 없으면 등록
    if not w3 in dic[w1][w2]: dic[w1][w2][w3] = 0 # w3 단어가 w2 블록 안에 없으면 등록 후 0으로 초기화
    dic[w1][w2][w3] += 1   # w3 단어가 나왔기 때문에 +1
    
    if word == '.' or word == '?':  # 마침표를 만나면 tmp 초기화
        tmp = ['@']

print(dic)

{'@': {'이번': {'에': 1}}, '이번': {'에': {'캘리포니아': 1}}, '에': {'캘리포니아': {'에': 1}, '산불': {'난': 1}}, '캘리포니아': {'에': {'산불': 1}}, '산불': {'난': {'거': 1}}, '난': {'거': {'보셨어요': 1}}, '거': {'보셨어요': {'네': 1}}, '보셨어요': {'네': {'봤어요': 1}}, '네': {'봤어요': {'ㅜㅜ': 1}}, '봤어요': {'ㅜㅜ': {'일주일': 1}}, 'ㅜㅜ': {'일주일': {'넘게': 1}}, '일주일': {'넘게': {'진압': 1}}, '넘게': {'진압': {'하고': 1}}, '진압': {'하고': {'있다는데': 1}}, '하고': {'있다는데': {'걱정': 1}}, '있다는데': {'걱정': {'이네': 1}}, '걱정': {'이네': {'요': 1}}}


In [9]:
import json
pretty = json.dumps(dic, indent=2, ensure_ascii=False)
print(pretty)

{
  "@": {
    "이번": {
      "에": 1
    }
  },
  "이번": {
    "에": {
      "캘리포니아": 1
    }
  },
  "에": {
    "캘리포니아": {
      "에": 1
    },
    "산불": {
      "난": 1
    }
  },
  "캘리포니아": {
    "에": {
      "산불": 1
    }
  },
  "산불": {
    "난": {
      "거": 1
    }
  },
  "난": {
    "거": {
      "보셨어요": 1
    }
  },
  "거": {
    "보셨어요": {
      "네": 1
    }
  },
  "보셨어요": {
    "네": {
      "봤어요": 1
    }
  },
  "네": {
    "봤어요": {
      "ㅜㅜ": 1
    }
  },
  "봤어요": {
    "ㅜㅜ": {
      "일주일": 1
    }
  },
  "ㅜㅜ": {
    "일주일": {
      "넘게": 1
    }
  },
  "일주일": {
    "넘게": {
      "진압": 1
    }
  },
  "넘게": {
    "진압": {
      "하고": 1
    }
  },
  "진압": {
    "하고": {
      "있다는데": 1
    }
  },
  "하고": {
    "있다는데": {
      "걱정": 1
    }
  },
  "있다는데": {
    "걱정": {
      "이네": 1
    }
  },
  "걱정": {
    "이네": {
      "요": 1
    }
  }
}


## 데이터

위의 책에서는 소설 토지로 사전을 만들었지만, 챗봇에는 어울리지 않아 아래 사이트에서 제공하는 SNS 대화 텍스트 파일을 이용했습니다.

<img width="600px" src="https://user-images.githubusercontent.com/8787919/203877544-822fcb64-55ce-4521-a3ac-ba6f597d309b.png">

데이터 파일만 98,654개이고 하나의 파일은 대략 20에서 30줄 정도의 문장으로 이루어져 있습니다. 이것을 사전으로 변환하는데 제 M1 맥북에어로 이틀을 꼬박 돌렸는데도 절반도 하지 못했고, 메모리가 부족해서인지 속도가 점점 더 느려져 결국 중단했습니다. 그래서 10,000개 파일만 변환했습니다. 

## 문장 생성

In [15]:
import random

def word_choice(select):
    keys = select.keys()
    return random.choice(list(keys))

input_list = ['@', '산불', '안녕']

for input_text in input_list:
    if not input_text in dic: break
    sentence = []
    if input_text != '@': sentence.append(input_text)
    top = dic[input_text]
    w1 = word_choice(top)
    w2 = word_choice(top[w1])
    sentence.append(w1)
    sentence.append(w2)
    while True:
        if w1 in dic and w2 in dic[w1]:
            w3 = word_choice(dic[w1][w2])
        else: w3 = ''
        sentence.append(w3)
        if w3 == '\n' or w3 == '' or w3 == '.' or w3 == '?': break
        w1, w2 = w2, w3
    result = ' '.join(sentence)
    print(result)

이번 에 캘리포니아 에 산불 난 거 보셨어요 네 봤어요 ㅜㅜ 일주일 넘게 진압 하고 있다는데 걱정 이네 요 
산불 난 거 보셨어요 네 봤어요 ㅜㅜ 일주일 넘게 진압 하고 있다는데 걱정 이네 요 


## word2vec 이용 사용자가 입력한 단어와 유사한 단어로 문장 생성 

위 코드는 사용자가 입력한 단어를 포함해 문장을 만듭니다. 마치 앵무새처럼 말이죠. 그래서 같은 단어를 사용하는 대신에 의미가 유사한 단어를 이용하는 것으로 바꿔보았습니다. 

사전 파일을 만들 때 사용한 데이터에서 명사를 추출하여 word2vec으로 학습시켰습니다. 사용자가 입력한 단어 중에서 명사를 뽑아 word2vec으로 유사한 단어를 찾은 뒤 그 단어를 사전에 키로 넣어 문장을 만들었습니다. 

In [27]:
from gensim.models import Word2Vec
w2v = Word2Vec(sentences=word_list, vector_size=200, window=4, hs=1, min_count=2, sg=1)
print('corpus count:', w2v.corpus_count)
print('corpus total words:', w2v.corpus_total_words)

corpus count: 22
corpus total words: 44
